# Segmenting and Clustering Neighborhoods in Toronto, Canada

*Library import section:*

In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

import requests
from bs4 import BeautifulSoup # library for scraping


print('Libraries imported.')

Libraries imported.


# Exercise 1

*Source scrape URL:*

In [3]:
source = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

*Import and transform the source URL, to a dataframe with given bounderies:*

In [30]:
#scrape Wiki
res = requests.get(source)
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]


#create list of the scraped inforation
postcode = df["Postcode"].tolist()
borough = df["Borough"].tolist()
neighborhood = df["Neighbourhood"].tolist()

#create DataFrame from Scraped table
sourceDF = pd.DataFrame(list(zip(postcode,borough,neighborhood)), columns =['PostalCode', 'Borough','Neighbourhood']) 
print(sourceDF.shape)

#remove lines where 'Borough' is Not Assigned
sourceDF = sourceDF[sourceDF.Borough != 'Not assigned']
sourceDF.index = np.arange(0, len(sourceDF))

#if Neighbourhood is 'Not assigned' replace value with whatis stored in 'Borough'
sourceDF['Neighbourhood'] = np.where(sourceDF['Neighbourhood'] == 'Not assigned', sourceDF['Borough'], sourceDF['Neighbourhood'])

#Group by to combine Neighbourhood in the same 'PostalCode' and 'Borough'
sourceDF = sourceDF.groupby (['PostalCode', 'Borough'], as_index=False).agg(lambda x: ', '.join(x))

(287, 3)


In [5]:
sourceDF.head(20)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
sourceDF.shape

(103, 3)

# Exercise 2

In [13]:
!wget -q -O 'Toronto_long_lat_data.csv'  http://cocl.us/Geospatial_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [31]:
lonlatDF = pd.read_csv('Toronto_long_lat_data.csv')
lonlatDF.head()
lonlatDF.columns=['PostalCode','Latitude','Longitude']
lonlatDF.head(10)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [23]:
sourceDF.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [32]:
torontoDF = pd.merge(sourceDF,
                 lonlatDF[['PostalCode','Latitude', 'Longitude']],
                 on='PostalCode')
torontoDF.head(20)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# Exercise 3

_Explore and cluster the neighborhoods in Toronto. We are replicating the same analysis we did to the New York City data._

In [33]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geographiclib-1.50   | 34 KB     | ##################################### | 100% 
geopy-1.20.0         | 57 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  en

In [34]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitudeToronto = location.latitude
longitudeToronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitudeToronto, longitudeToronto))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [36]:
#json for toronto in case needed: http://adamw523.com/toronto-geojson/

torontoMap = folium.Map(location=[latitudeToronto, longitudeToronto], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(torontoDF['Latitude'], torontoDF['Longitude'], torontoDF['Borough'], torontoDF['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(torontoMap)  
    
torontoMap

In [37]:
#Credentials omited
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0N455GT5MN0KZIZGGCIQBJEL2AF0HMZIJVD0KIPM0IZYQEGR
CLIENT_SECRET:LOYR0HWYOFCXFBOVYBB2BU4XHXNBVZUX2ENH3NWNJGCMUNXF


In [38]:
# defining radius  of venues to get
radius=500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    listVenues=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        listVenues.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearbyVenues = pd.DataFrame([item for venue_list in listVenues for item in venue_list])
    nearbyVenues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearbyVenues)

In [39]:
torontoVenues = getNearbyVenues(names=torontoDF['Neighbourhood'],
                                   latitudes=torontoDF['Latitude'],
                                   longitudes=torontoDF['Longitude']
                                  )
                                  

#quick view of our DF                                  
torontoVenues.head(10)

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
5,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
6,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
7,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
8,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection
9,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot


In [40]:
torontoVenues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",9,9,9,9,9,9
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Downsview North, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,57,57,57,57,57,57


_Analysing Each Neighborhood_

In [41]:
# one hot encoding
torontoOnehot = pd.get_dummies(torontoVenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
torontoOnehot['Neighbourhood'] = torontoVenues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [torontoOnehot.columns[-1]] + list(torontoOnehot.columns[:-1])

In [42]:
torontoOnehot.head()
torontoOnehot.shape

(2245, 270)

_What about the frequency of each venue category?_

In [43]:
torontoGrouped = torontoOnehot.groupby('Neighbourhood').mean().reset_index()
torontoGrouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03,0.0,0.0,0.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.03,0.0,0.04,0.0,0.0,0.0,0

_What about the top most common venues per neighborhood?_

In [45]:
# let's say top 3 venues
numTP = 3

for hood in torontoGrouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = torontoGrouped[torontoGrouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(numTP))
    print('\n')
torontoOnehot.shape

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.05
2   Steakhouse  0.04


----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
         venue  freq
0   Playground  0.33
1  Coffee Shop  0.33
2         Park  0.33


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0           Pizza Place  0.11
1         Grocery Store  0.11
2  Fast Food Restaurant  0.11


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.25
1           Pub  0.12
2  Skating Rink  0.12


----Bathurst Manor, Downsview North, Wilson Heights----
         venue  freq
0  Coffee Shop  0.11
1     Pharmacy  0.05
2   Restaurant  0.05


----Bayview Village----
                 venue  freq
0             

(2245, 270)

_We can no put the above results into a dataframe_

In [47]:
# here we have a function to create a dataframe with sorted rows
def return_most_common_venues(row, numTP):
    rowCategories = row.iloc[1:]
    rowCategoriesSorted = rowCategories.sort_values(ascending=False)
    
    return rowCategoriesSorted.index.values[0:numTP]

In [48]:
# Now let's create the new dataframe and display the top 5 venues for each neighborhood.
numTP = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(numTP):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoodsVenuesSorted = pd.DataFrame(columns=columns)
neighbourhoodsVenuesSorted['Neighbourhood'] = torontoGrouped['Neighbourhood']

for ind in np.arange(torontoGrouped.shape[0]):
    neighbourhoodsVenuesSorted.iloc[ind, 1:] = return_most_common_venues(torontoGrouped.iloc[ind, :], numTP)

neighbourhoodsVenuesSorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Thai Restaurant
1,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Ethiopian Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Coffee Shop,Drugstore,Dim Sum Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fried Chicken Joint,Sandwich Place,Discount Store,Pizza Place
4,"Alderwood, Long Branch",Pizza Place,Skating Rink,Coffee Shop,Pub,Sandwich Place


### Toronto Cluster Neighborhoods

In [50]:
# set number of clusters
kclusters = 3

torontoGroupedClustering = torontoGrouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(torontoGroupedClustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 
# to change use .astype()

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 1, 2, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1], dtype=int32)

_And now we create a new dataframe that includes the cluster as well as the top 5 venues for each neighborhood._

In [51]:
# add clustering labels
neighbourhoodsVenuesSorted.insert(0, 'Cluster_Labels', kmeans.labels_)

torontoMerged = torontoDF
torontoMerged = torontoMerged.join(neighbourhoodsVenuesSorted.set_index('Neighbourhood'), on='Neighbourhood')


torontoMerged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0.0,Bar,History Museum,Yoga Studio,Dumpling Restaurant,Discount Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Pizza Place,Medical Center,Intersection,Rental Car Location,Mexican Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Eastern European Restaurant,Discount Store,Dog Run
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Bakery,Bank,Hakka Restaurant,Caribbean Restaurant,Thai Restaurant


In [52]:
# We can quickly see some data might not be available, so we should clean it
torontoMerged=torontoMerged.dropna()
torontoMerged['Cluster_Labels'] = torontoMerged.Cluster_Labels.astype(int)

In [54]:
# create map
clustersMap = folium.Map(location=[latitudeToronto, longitudeToronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markersColors = []
for lat, lon, poi, cluster in zip(torontoMerged['Latitude'], torontoMerged['Longitude'], torontoMerged['Neighbourhood'], torontoMerged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(clustersMap)

In [55]:
clustersMap

_Examine Clusters:_

In [56]:
#Cluster 1
torontoMerged.loc[torontoMerged['Cluster_Labels'] == 0, torontoMerged.columns[[1] + list(range(5, torontoMerged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
32,North York,2,Home Service,Baseball Field,Food Truck,Dim Sum Restaurant,Discount Store
63,Central Toronto,2,Home Service,Garden,Dessert Shop,Diner,Discount Store
91,Etobicoke,2,Home Service,Baseball Field,Dim Sum Restaurant,Discount Store,Dog Run
97,North York,2,Baseball Field,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run


In [57]:
#Cluster 2
torontoMerged.loc[torontoMerged['Cluster_Labels'] == 1, torontoMerged.columns[[1] + list(range(5, torontoMerged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
14,Scarborough,1,Playground,Park,Coffee Shop,Drugstore,Dim Sum Restaurant
20,North York,1,Park,Cafeteria,Yoga Studio,Drugstore,Diner
23,North York,1,Bank,Park,Convenience Store,Yoga Studio,Dumpling Restaurant
25,North York,1,Food & Drink Shop,Fireworks Store,Park,Yoga Studio,Discount Store
30,North York,1,Airport,Park,Electronics Store,Yoga Studio,Drugstore
40,East York,1,Park,Coffee Shop,Convenience Store,Yoga Studio,Dumpling Restaurant
44,Central Toronto,1,Bus Line,Park,Swim School,Yoga Studio,Drugstore
48,Central Toronto,1,Tennis Court,Trail,Playground,Park,Dessert Shop
50,Downtown Toronto,1,Park,Trail,Playground,Yoga Studio,Dim Sum Restaurant
64,Central Toronto,1,Jewelry Store,Park,Sushi Restaurant,Trail,Drugstore


In [58]:
#Cluster 3
torontoMerged.loc[torontoMerged['Cluster_Labels'] == 2, torontoMerged.columns[[1] + list(range(5, torontoMerged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
32,North York,2,Home Service,Baseball Field,Food Truck,Dim Sum Restaurant,Discount Store
63,Central Toronto,2,Home Service,Garden,Dessert Shop,Diner,Discount Store
91,Etobicoke,2,Home Service,Baseball Field,Dim Sum Restaurant,Discount Store,Dog Run
97,North York,2,Baseball Field,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run
